In [85]:
from itertools import combinations, product

In [86]:

def rook_placement(partition,k):
    #For a partition = (board,nb_vertices) and k = number of rooks, returns all the possible rook placements of k rooks
    partition,_=partition
    placements = []
    for subset in Combinations(partition.cells(), k):
        rows = [r for (r, c) in subset]
        cols = [c for (r, c) in subset]
        if len(set(rows)) == len(rows) and len(set(cols)) == len(cols):
            placements.append(subset)
    return placements

In [104]:
def size_min_square(partition):
    #For a board, returns the size of the minimum square that can contain this board (thus the minimum number of vertices)
    maxi=0
    for i in range(len(partition)):
        maxi=max(partition[i]+i+1,maxi)
    return maxi

def linked_path(partition,rooks):
    #For a partition = (board,nb_vertices) and rooks = (array of cells occupied by rooks)
    #Return an array, each case is a array of cells representing a linked path in increasing order of rank 
    #The size of the big array is the number of linked paths and the size of each small array is the rank of the path
    partition,s=partition
    cols_rook=[-1]*s
    cols_length=partition.conjugate()
    cols_seen=[False]*s
    cols_rank=[[] for _ in range(s)]
    for i,j in rooks:
        cols_rook[j]=i
    for i in range(s):
        if not(cols_seen[i]):
            cols_seen[i]=True
            cols_rank[i].append((s-i-1,0))
            r=1
            j=cols_rook[i]
            cols_rank[i].append((j,r))
            if j!=-1:
                j=s-j-1                              
            while j!=-1:
                    r+=1
                    cols_seen[j]=True
                    cols_rank[j].append((cols_rook[j],r))
                    if cols_rook[j]!=-1:
                        i,j=j,s-cols_rook[j]-1
                    else:
                        i,j=j,-1
                    
    return cols_rank
                    
            
    

In [88]:
gamma=(Partition([5,5,3,3,1]),size_min_square(Partition([5,5,3,3,1])))
rooks=[(4,0),(2,2),(1,3)]
cols_rank=linked_path(gamma,rooks)
print(cols_rank)
print(gamma)

[[(6, 0), (4, 1)], [(5, 0), (-1, 1)], [(2, 2)], [(3, 0), (1, 1)], [(-1, 3)], [(-1, 2)], [(0, 0), (-1, 1)]]
([5, 5, 3, 3, 1], 7)


In [89]:
def type_linked(cols_rank):
    #For cols_ranks = an array of linked paths, return the sorted array of the ranks of the paths
    rank_type = []
    for tab in cols_rank:
        for i,j in tab:
            if i==-1:
                rank_type.append(j)
    return sorted(rank_type,reverse=True)

In [90]:
def find_rook_row(cols_rank,row):
    #For cols_rank = array of linked paths and a given row
    #Look for a rook in this row, if it exists, it returns the index of the column and the rank of the rook (otherwise return -1,+inf)
    for i in range(len(cols_rank)):
        for j,rank in cols_rank[i]:
            if j==row:
                return i,rank
    return -1,oo
    
def free_cells(partition,cols_rank):
    #For partition = (board,nb_vertices) and cols_rank = array of linked paths, return the array of free cells
    partition,_=partition
    set_free_cells = []
    for i,j in partition.cells():
        for row,r1 in cols_rank[j]:
            if i>row:
                jprime,r2=find_rook_row(cols_rank,i)
                if jprime>j and r1>=r2:
                    set_free_cells.append((i,j))
                elif jprime<j and r1>r2:
                    set_free_cells.append((i,j))
    return set_free_cells


In [91]:
def n(mu):
    #mu=the array of the ranks of the paths
    total=0
    for i in range(len(mu)):
        total+=mu[i]*i
    return total

def area(gamma):
    #gamma=(board,nb_vertices)
    gamma,s=gamma
    s-=1
    total=0
    for i in range(s):
        if i<len(gamma):
            total+=(s-i-gamma[i])
        else:
            total+=(s-i)
    return total

def statistic(partition,rook):
    #gamma=(board,nb_vertices) and rook=rook placement
    ranks=linked_path(partition,rook)
    mu=type_linked(ranks)
    fc = len(free_cells(partition,ranks))
    #print(area(gamma),fc,n(mu))
    return area(gamma)+fc-n(mu)        
    

In [92]:
def enumerate_rook_placement(gamma,stat):
    #gamma=(board,nb_vertices), stat=number we want for the statistic of the placements
    correct_placement=[]
    max_nb_rook = min(len(gamma[0]),gamma[0][0])
    for k in range(0,max_nb_rook+1):
        all_placements = rook_placement(gamma,k)
        for placement in all_placements:
            s=statistic(gamma,placement)
            if s==stat:
                correct_placement.append((gamma,placement))
    return correct_placement

In [93]:
def enumerate_k_rook_placement(gamma,k,stat):
    #gamma=(board,nb_vertices), k=number of rooks and stat=number we want for the statistic of the placements
    correct_placement=[]
    max_nb_rook = min(len(gamma[0]),gamma[0][0])
    all_placements = rook_placement(gamma,k)
    for placement in all_placements:
        s=statistic(gamma,placement)
        if s==stat:
            correct_placement.append((gamma,placement))
    return correct_placement

In [94]:
def enumerate_k_rook_placement_fixed_rank(gamma,k,rank,stat):
    #gamma=(board,nb_vertices), k=number of rooks,rank=array of the rank of the placement
    #stat=number we want for the statistic of the placements
    correct_placement = []
    max_nb_rook = min(len(gamma[0]),gamma[0][0])
    all_placements = rook_placement(gamma,k)
    for placement in all_placements:
        s=statistic(gamma,placement)
        if s==stat and type_linked(linked_path(gamma,placement)) == rank:
            correct_placement.append((gamma,placement))
    return correct_placement

In [95]:
def pretty_print_rook_placement(gamma,placement,rank):
    #gamma=(board,nb_vertices), placement=array of cells (rook placement) and rank=array of linked paths
    board=[['-']*i for i in gamma[0]]
    for i,j in placement:
        board[i][j]='*'
    fc=free_cells(gamma,rank)
    for i,j in fc:
        board[i][j]='o'
    for row in board:
        print(*row)
    print()

def pretty_print(placements):
    #array of tuple: placement+rank
    for g,p in placements:
        pretty_print_rook_placement(g,p,linked_path(g,p))

In [96]:
pretty_print(enumerate_rook_placement(gamma,0))

o o o o o
o o o o o
o o o
o o o
o

- o o o o
* - - - -
o o o
o o o
o

- o o o o
- o o o o
- o o
* - -
o

- * - - -
* - - - -
o o o
o o o
o

- - o o o
- * - o -
- o o
* - -
o

- o o - o
- - - * -
- o o
* - -
o

- - o o o
- - o o o
- * -
* - -
o

- * - - -
- - - * -
- o o
* - -
o

- - * o -
- * - o -
- o o
* - -
o

- - - o o
- - * o o
- * -
* - -
o

- - o - o
- - - * -
- * -
* - -
o

- - * - o
- - - * -
- * -
* - -
o

- - - - *
- - - * -
- * -
* - -
o



In [97]:
rook1=[(4,0),(2,2),(1,3)]
rook2=[(4,0),(1,3)]
rook3=[(4,0)]
print(statistic(gamma,rook1))
print(statistic(gamma,rook2))
print(statistic(gamma,rook3))

3
2
1


In [98]:
gamma=(Partition([7,6,6,3,3,3,1]),9)
rook1=[(0,5),(1,4),(2,1),(3,0)]
rook2=[(0,3),(1,2),(2,1),(5,0)]
rook3=[(0,5),(1,4),(2,3),(3,0)]
print(statistic(gamma,rook1))
print(statistic(gamma,rook2))
print(statistic(gamma,rook3))

5
1
7


In [99]:
gamma=(Partition([6,5,5,3,2]),8)
pretty_print(enumerate_rook_placement(gamma,0))

o o o o o o
o o o o o
o o o o o
o o o
o o

- o o o o o
- o o o o
* - - - -
o o o
o o

- - o o o o
- * - - -
* - - - -
o o o
o o

- - * - - o
- * - - -
* - - - -
o o o
o o



In [100]:
gamma=(Partition([10,10,8,7,7,5,5,2,2,1]),12)
enumerate_k_rook_placement_fixed_rank(gamma,6,[3,3,2,2,1,1],0)

[(([10, 10, 8, 7, 7, 5, 5, 2, 2, 1], 12),
  [(0, 3), (1, 2), (3, 6), (4, 5), (5, 1), (6, 0)]),
 (([10, 10, 8, 7, 7, 5, 5, 2, 2, 1], 12),
  [(0, 3), (1, 6), (3, 2), (4, 5), (5, 1), (6, 0)]),
 (([10, 10, 8, 7, 7, 5, 5, 2, 2, 1], 12),
  [(0, 3), (1, 8), (2, 2), (3, 1), (4, 5), (6, 0)]),
 (([10, 10, 8, 7, 7, 5, 5, 2, 2, 1], 12),
  [(0, 6), (1, 5), (3, 3), (4, 2), (5, 1), (6, 0)]),
 (([10, 10, 8, 7, 7, 5, 5, 2, 2, 1], 12),
  [(0, 7), (1, 5), (2, 3), (3, 2), (4, 1), (6, 0)]),
 (([10, 10, 8, 7, 7, 5, 5, 2, 2, 1], 12),
  [(1, 3), (2, 2), (3, 6), (4, 5), (5, 1), (6, 0)]),
 (([10, 10, 8, 7, 7, 5, 5, 2, 2, 1], 12),
  [(1, 6), (2, 3), (3, 2), (4, 5), (5, 1), (6, 0)])]

In [18]:
gamma=(Partition([10,10,7,7,7,5,5]),12)
enumerate_k_rook_placement_fixed_rank(gamma,6,[3,3,2,2,1,1],0)

- - - * - - - o o -
- - * - - - - o o -
- - o o o - -
- - - - - - *
- - - - - * -
- * - - -
* - - - -


- - - * - - - o o -
- - - - - - * o - -
- - - o o - o
- - * - - - o
- - - - - * -
- * - - -
* - - - -


- - - * - - - o - o
- - - - - - - o * -
- - * - - - -
- * - - - - -
- - - - - * -
- o o o o
* - - - -


- - - - - - * - - -
- - - - - * - - - -
- - - - o o o
- - - * - o o
- - * - - o o
- * - - -
* - - - -


- - - - - - - * - -
- - - - - * - - - -
- - - * - o -
- - * - - o -
- * - - - o -
- o o o o
* - - - -


- - - - o - - o o o
- - - * - - - o o o
- - * - - - -
- - - - - - *
- - - - - * -
- * - - -
* - - - -


- - - - o - - o o o
- - - - - - * o - -
- - - * - - o
- - * - - - o
- - - - - * -
- * - - -
* - - - -




In [19]:
x = SemistandardTableaux([6,4,2],max_entry=4)
for tab in x.list():
    rank=[0]*4
    for i in range(4):
        for k in range(3):
            rank[i]+=tab[k].count(i+1)
    if rank==[5,2,3,2]:
        tab.pp()
        print()

  1  1  1  1  1  2
  2  3  3  3
  4  4

  1  1  1  1  1  2
  2  3  3  4
  3  4

  1  1  1  1  1  3
  2  2  3  3
  4  4

  1  1  1  1  1  3
  2  2  3  4
  3  4

  1  1  1  1  1  3
  2  2  4  4
  3  3

  1  1  1  1  1  4
  2  2  3  3
  3  4

  1  1  1  1  1  4
  2  2  3  4
  3  3



In [46]:
gamma=(Partition([10,10,7,7,7,5,5]),12)
pretty_print(enumerate_k_rook_placement_fixed_rank(gamma,6,[3,3,2,2,1,1],0))

- - - * - - - o o -
- - * - - - - o o -
- - o o o - -
- - - - - - *
- - - - - * -
- * - - -
* - - - -

- - - * - - - o o -
- - - - - - * o - -
- - - o o - o
- - * - - - o
- - - - - * -
- * - - -
* - - - -

- - - * - - - o - o
- - - - - - - o * -
- - * - - - -
- * - - - - -
- - - - - * -
- o o o o
* - - - -

- - - - - - * - - -
- - - - - * - - - -
- - - - o o o
- - - * - o o
- - * - - o o
- * - - -
* - - - -

- - - - - - - * - -
- - - - - * - - - -
- - - * - o -
- - * - - o -
- * - - - o -
- o o o o
* - - - -

- - - - o - - o o o
- - - * - - - o o o
- - * - - - -
- - - - - - *
- - - - - * -
- * - - -
* - - - -

- - - - o - - o o o
- - - - - - * o - -
- - - * - - o
- - * - - - o
- - - - - * -
- * - - -
* - - - -



In [ ]:
gamma=(Partition([3,3,3]),6)
enumerate_rook_placement(gamma,0)

In [ ]:
gamma=(Partition([4,3,3,1]),6)
enumerate_rook_placement(gamma,0)

In [ ]:
gamma=(Partition([2,2,2,2]),6)
enumerate_rook_placement(gamma,0)

In [ ]:
gamma=(Partition([4,3,2,2]),6)
enumerate_rook_placement(gamma,0)

In [ ]:
gamma=(Partition([7,7,7,7,3,3,3,3]),11)
enumerate_rook_placement(gamma,0)

In [54]:
gamma=(Partition([3,2]),4)
pretty_print(enumerate_rook_placement(gamma,0))

o o o
o o

* - -
o o

- o o
* -

- * o
* -

- - *
* -



In [76]:
def size_components(gamma):
    size=[]
    size_component=1
    for i in range(gamma[1]-2,-1,-1):
        if i>=len(gamma[0]):
            size_component+=1
        else:
            if gamma[0][i]==gamma[1]-i-1:
                size.append(size_component)
                size_component=1
            else:
                size_component+=1
    size.append(size_component)
    return size


def cumul_tab(tab):
    cumul=[0]*len(tab)
    cumul[0]=tab[0]
    for i in range(1,len(tab)):
        cumul[i]=cumul[i-1]+tab[i]
    return cumul

    
def ssyt_from_placement(gamma,placement):
    paths=linked_path(gamma,placement)
    nb_rows=type_linked(paths)[0]
    ssyt=[[] for _ in range(nb_rows)]
    current_component=1
    component=cumul_tab(size_components(gamma))
    for i in range(gamma[1]-1,-1,-1):
        if gamma[1]-i>component[current_component-1]:
            current_component+=1
        c,r = find_rook_row(paths,i)
        ssyt[r].append(current_component)
    return ssyt

def pp_ssyt(tab):
    for t in tab:
        print(*t)
    print()

In [78]:
placements=enumerate_rook_placement(gamma,0)
for g,p in placements:
    pp_ssyt(ssyt_from_placement(partition,p))
    pretty_print_rook_placement(g,p,linked_path(g,p))

1 1 2 3

o o o
o o

1 1 2
3

* - -
o o

1 1 3
2

- o o
* -

1 1
2 3

- * o
* -

1 1
2
3

- - *
* -



In [106]:
gamma=(Partition([10,10,7,7,7,5,5]),12)
placements=enumerate_rook_placement(gamma,0)
for g,p in placements:
    pretty_print_rook_placement(g,p,linked_path(g,p))                       
    pp_ssyt(ssyt_from_placement(g,p))

o o o o o o o o o o
o o o o o o o o o o
o o o o o o o
o o o o o o o
o o o o o o o
o o o o o
o o o o o

1 1 1 1 1 2 2 3 3 3 4 4

- o o o o o o o o o
* - - - - - - - - -
o o o o o o o
o o o o o o o
o o o o o o o
o o o o o
o o o o o

1 1 1 1 1 2 2 3 3 3 4
4

- o o o o o o o o o
- o o o o o o o o o
- o o o o o o
- o o o o o o
* - - - - - -
o o o o o
o o o o o

1 1 1 1 1 2 2 3 3 4 4
3

- o o o o o o o o o
- o o o o o o o o o
- o o o o o o
- o o o o o o
- o o o o o o
- o o o o
* - - - -

1 1 1 1 1 2 3 3 3 4 4
2

- * - - - - - - - -
* - - - - - - - - -
o o o o o o o
o o o o o o o
o o o o o o o
o o o o o
o o o o o

1 1 1 1 1 2 2 3 3 3
4 4

- - o o o o o o o o
- * - - - - - o - -
- o o o o o o
- o o o o o o
* - - - - - -
o o o o o
o o o o o

1 1 1 1 1 2 2 3 3 4
3 4

- - o o o o o o o o
- * - - - o - - - -
- o o o o o o
- o o o o o o
- o o o o o o
- o o o o
* - - - -

1 1 1 1 1 2 3 3 3 4
2 4

- o o o o - o o o o
- - - - - * - - - -
- o o o o o o
- o o o o o o
- o o o o o o
- o o o o
* - - - -

1

In [ ]:
#I have an algorithm to recover a rook placement from a SSYT.